# Cookbook

Here we provide a set of recipes detailing the functionality available in our set of packages

## Including the libraries

In [1]:
from converting_cats_in_cats_of_bins import * #To convert any halo catalog according to bins of mass
from colossus.cosmology import cosmology #To compute bias
from colossus.lss import bias #To compute bias
from cosmo import cosmo #to set the cosmology
from code_options import code_parameters #to set the cosmological parameters
from create_grids_from_xyz_cats_beta import * #to build the cats
from MTPK_estimate_beta import * #to estimate the spectra

## Spliting the catalogs in bins

One of the main characteristics of the `MTPK` package is to compute the power spectra for different tracers. Here we present a library to split them according to halo mass. The parameters can be given according to:

In [2]:
#This method print the parameters from this class
converting_cats_in_cats_of_bins().params_print()

cats = ['data/ExSHalos/L128_000_halos.dat', 'data/ExSHalos/L128_001_halos.dat', 'data/ExSHalos/L128_002_halos.dat', 'data/ExSHalos/L128_003_halos.dat']
m_col = 6
x_col = 0
y_col = 1
z_col = 2
m_min = 316227766016.83795
m_max = 10000000000000
n_bins = 3
skiprows = 1
V = 2097152.0
path_to_save = data/ExSHalos/


''

Then, you can instantiate the params according to your choices. For example:

In [3]:
sp_bins = converting_cats_in_cats_of_bins()
sp_bins

In [4]:
MF = sp_bins.to_bins()
MF

Catalogs created!


array([0.01251531, 0.00373197, 0.0007205 ])

The central masses per bin are given using the `method`:

In [5]:
m_ctrs = sp_bins.central_masses()
m_ctrs

[658113883008.419, 2081138830084.1897, 6581138830084.189]

To compute an estimative for the bias of each halo bin we can use the `Colossus` library:

In [6]:
cosmology.setCosmology('planck18')
b = []
for i in range( len(m_ctrs) ):
    b.append( bias.haloBias(m_ctrs[i], model = 'tinker10', z = 1, mdef = 'vir') )
b

[1.2086210355250606, 1.5027890453729722, 1.9830274528457887]

## Setting the cosmology

The cosmological parameters have default values according to [Planck 2018](https://arxiv.org/abs/1807.06209). You can check the individual values just using

In [7]:
#Printing the default cosmology
cosmo().cosmo_print()

h = 0.678
Omega0_b = 0.048206
Omega0_cdm = 0.2589
Omega0_m = 0.307106
Omega0_k = 0.0
Omega0_DE = 0.692894
A_s = 2.1867466842075255e-09
ln10e10AsA = 3.085
n_s = 0.96
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrowcentral = 1e-05
zcentral = 1.0
c_light = 299792.458


''

The first argument of all the functions in this suite uses the set of cosmological parameters, then, set the cosmology is the first thing that you have to do in order to compute the power spectra.

### Setting the cosmology as the fiducial one

Otherwise you can change the value of the parameters simply giving the parameter and its new value. For instance: `cosmo(h = 0.678, n_s = 0.96)`

You can still define the parameter `matgrow` according to two different methods:

In [8]:
mg_evol = cosmo().f_evolving(0)
mg_phe = cosmo().f_phenomenological()
mg_evol, mg_phe

(0.5252511834620611, 0.5252511834620611)

In [9]:
my_cosmology = cosmo(matgrowcentral = mg_evol)
my_cosmology.cosmo_print()

h = 0.678
Omega0_b = 0.048206
Omega0_cdm = 0.2589
Omega0_m = 0.307106
Omega0_k = 0.0
Omega0_DE = 0.692894
A_s = 2.1867466842075255e-09
ln10e10AsA = 3.085
n_s = 0.96
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrowcentral = 0.5252511834620611
zcentral = 1.0
c_light = 299792.458


''

## Setting the code options

The second argument of all the functions in this suite uses the set of code options. These options define exactly what kind of data you are going to compute the power spectra and which are the spectra that you are obtaining. The fiducial option is given by:

In [10]:
#Defining the code options
my_code_options = code_parameters()
my_code_options.parameters_print()

method = both
mas_method = CIC
use_kdip_phys = False
kdip_phys = 0.005
multipoles_order = 4
do_cross_spectra = True
use_padding = False
padding_length = [10, 10, 10]
use_theory_spectrum = False
theory_spectrum_file = theory_spectrum_file.txt
use_mask = False
mask_filename = mask.hdf5
mass_fun = [0.0156  0.00443 0.00143]
halo_bias = [1.572 1.906 2.442]
nhalos = 3
n_maps = 3
cell_size = 1.0
n_x = 128
n_y = 128
n_z = 128
n_x_orig = -64.0
n_y_orig = -64.0
n_z_orig = 10000.0
sel_fun_data = False
sel_fun_file = sel_fun-N128_halos.hdf5
kmin_bias = 0.05
kmax_bias = 0.15
kph_central = 0.1
dkph_bin = 0.01
use_kmin_phys = False
kmin_phys = 0.05
use_kmax_phys = True
kmax_phys = 1.0
whichspec = 1
use_cell_low_count_thresh = False
cell_low_count_thresh = 0.0
mult_sel_fun = 1.0
shift_sel_fun = 0.0
k_min_CAMB = 0.0001
k_max_CAMB = 1.0


''

### Setting different options

You can define different code options. However, some parameters are inherent from the dataset that you are using, so, pay attentio on that:

In [11]:
#Defining the code options
my_code_options = code_parameters(method = 'FKP', multipoles_order = 4, 
                                  n_maps = 4,
                                  nhalos = 3,
                                  mass_fun = MF,
                                  halo_bias = np.array(b))
my_code_options.parameters_print()

method = FKP
mas_method = CIC
use_kdip_phys = False
kdip_phys = 0.005
multipoles_order = 4
do_cross_spectra = True
use_padding = False
padding_length = [10, 10, 10]
use_theory_spectrum = False
theory_spectrum_file = theory_spectrum_file.txt
use_mask = False
mask_filename = mask.hdf5
mass_fun = [0.01251531 0.00373197 0.0007205 ]
halo_bias = [1.20862104 1.50278905 1.98302745]
nhalos = 3
n_maps = 4
cell_size = 1.0
n_x = 128
n_y = 128
n_z = 128
n_x_orig = -64.0
n_y_orig = -64.0
n_z_orig = 10000.0
sel_fun_data = False
sel_fun_file = sel_fun-N128_halos.hdf5
kmin_bias = 0.05
kmax_bias = 0.15
kph_central = 0.1
dkph_bin = 0.01
use_kmin_phys = False
kmin_phys = 0.05
use_kmax_phys = True
kmax_phys = 1.0
whichspec = 1
use_cell_low_count_thresh = False
cell_low_count_thresh = 0.0
mult_sel_fun = 1.0
shift_sel_fun = 0.0
k_min_CAMB = 0.0001
k_max_CAMB = 1.0


''

## Creating the catalogs

You can create the cats using `create_grids_from_xyz_cats_beta`

In [12]:
input_filename = 'ExSHalos'
filenames_catalogs = 'data/ExSHalos/seed*.dat'
Ncats = 4
Ntracers = 3
use_redshifts = False
col_x = 0                                                                                              
col_y = 1                                                                                              
col_z = 2
x_cat_min , y_cat_min , z_cat_min = 0.0 , 0.0 , 0.0
x_cat_max , y_cat_max , z_cat_max = 128.0 , 128.0 , 128.0
mask_redshift = False
grid_method = 2
save_mask = False
save_mean_sel_fun = False

In [13]:
create_grids_from_xyz_cats(my_cosmology, my_code_options, 
                           input_filename, 
                           filenames_catalogs, Ncats, Ntracers, 
                           use_redshifts, col_x, col_y, col_z, x_cat_min , 
                           y_cat_min, z_cat_min, x_cat_max, y_cat_max, 
                           z_cat_max, mask_redshift, grid_method,    
                           save_mask, save_mean_sel_fun)


Will load maps stored in files:
[['data/ExSHalos/seed0_bin0.dat' 'data/ExSHalos/seed0_bin1.dat'
  'data/ExSHalos/seed0_bin2.dat']
 ['data/ExSHalos/seed1_bin0.dat' 'data/ExSHalos/seed1_bin1.dat'
  'data/ExSHalos/seed1_bin2.dat']
 ['data/ExSHalos/seed2_bin0.dat' 'data/ExSHalos/seed2_bin1.dat'
  'data/ExSHalos/seed2_bin2.dat']
 ['data/ExSHalos/seed3_bin0.dat' 'data/ExSHalos/seed3_bin1.dat'
  'data/ExSHalos/seed3_bin2.dat']]
Dimensions of the grids: n_x, n_y, n_z = 128 128 128

The actual catalog spans the ranges in x,y,z:
x: 0.0 --> 128.0
y: 0.0 --> 128.0
z: 0.0 --> 128.0


With the padding length, of  0 cells, the box will be filled with:
x: ( 0 * 0 , 128 , 0 *0)
y: ( 0 * 0 , 128 , 0 *0)
z: ( 0 * 0 , 128 , 0 *0)

Check: given the padding, your catalog should end at cartesian positions:
max(x) = 128.0
max(y) = 128.0
max(z) = 128.0

Origin (0,0,0) of box will be considered to be displaced from the observer @Earth
by these numbers of cells in each direction:    (This affects RSDs!)
n_x_ori

0

## Estimating the spectra

You can estimate the spectra using the function inside `MTPK_estimate_beta` code. The `handle_data` option follows for the name of the file where you are going to compute the spectra:

In [14]:
MTPK_estimate(my_cosmology, my_code_options, handle_data = "ExSHalos")



This is the Multi-tracer power spectrum estimator

Handle of this run (fiducial spectra, biases, etc.):  ExSHalos

Beggining CAMB calculations

Computing matter power spectrum for given cosmology...

0
CALLING CAMB NOW - HALOFIT
Time elapsed: 0.7499873638153076
.
Generating the k-space Grid...
.


/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Will use the N = 4  simulation-only maps contained in directory /home/natalidesanti/doutorado/MTPK_github/MTPK/maps/sims/ExSHalos
.
Geometry: (nx,ny,nz) = (128,128,128),  cell_size=1.0 h^-1 Mpc
Geometry including bounding box: (nx,ny,nz) = (128,128,128)
.
Using power spectrum from CAMB + HaloFit
.
----------------------------------
.
Will estimate modes up to k[h/Mpc] =  1.0000  in bins with Delta_k = 0.0234
.
----------------------------------
.

----------------------------------

Central physical k values where spectra will be estimated: 0.1
Initializing the k-binning matrix...
Done with k-binning matrices. Time cost:  0.346 s
Memory occupied by the binning matrix:  16727
Originally k_bar was defined as: ['0.0441', '0.0516', '0.0590']
The true mean of k for each bin is: ['0.0444', '0.0513', '0.0587']

----------------------------------

Now estimating the power spectra...
Starting power spectra estimation
Initializing traditional (FKP) estimation toolbox...
... done. Starting comput

0